# Task 1 Massive Data Processing

# Crypto Currency Price History

In [3]:
import csv
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('../data/raw/bitcoin_cash_price.csv')

In [7]:
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,"Sep 05, 2017",514.90,550.95,458.78,541.71,"338,978,000","8,527,100,000"
1,"Sep 04, 2017",608.26,608.26,500.75,517.24,"328,957,000","10,072,200,000"
2,"Sep 03, 2017",578.27,617.41,563.59,607.43,"344,862,000","9,574,520,000"
3,"Sep 02, 2017",621.96,642.05,560.58,575.90,"350,478,000","10,297,000,000"
4,"Sep 01, 2017",588.40,645.52,586.73,622.17,"393,839,000","9,740,460,000"


In [8]:
df.tail()

,Date,Open,High,Low,Close,Volume,Market Cap
40,"Jul 27, 2017",417.10,460.97,367.78,385.48,"533,207",-
41,"Jul 26, 2017",407.08,486.16,321.79,365.82,"1,784,640",-
42,"Jul 25, 2017",441.35,541.66,338.09,406.90,"524,908",-
43,"Jul 24, 2017",412.58,578.89,409.21,440.70,"190,952",-
44,"Jul 23, 2017",555.89,578.97,411.78,413.06,"85,013",-


In [ ]:
df['Volume'] = df['Volume'].apply(lambda x: float(str(x).replace(',',''))) 
df['Market Cap'] = df['Market Cap'].replace('-', 'NaN')
df['Market Cap'] = df['Market Cap'].apply(lambda x: float(str(x).replace(',',''))) 


In [ ]:
df.tail()

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

I try to fill in the missing value by interpolating the data

In [ ]:
#Plot data to see the relation between each column
import matplotlib.pyplot as plt
plt.figure(figsize=(25, 25))
O = df['Open']
H = df['High']
L = df['Low']
C = df['Close']
V = df['Volume']
MC = df['Market Cap']

plt.subplot(5,5,1)
plt.scatter(O, MC)
plt.title('Open vs Market Cap')

plt.subplot(5,5,2)
plt.scatter(H, MC)
plt.title('High vs Market Cap')

plt.subplot(5,5,3)
plt.scatter(L, MC)
plt.title('Low vs Market Cap')

plt.subplot(5,5,4)
plt.scatter(C, MC)
plt.title('Close vs Market Cap')

plt.subplot(5,5,5)
plt.scatter(V, MC)
plt.title('Volume vs Market Cap')

plt.show

From data plot above that can be seen Market Cap has linear value with Open. So then, I try interpolate the value of Market Cap with Open value.

In [ ]:
from sklearn import linear_model

model = linear_model.LinearRegression()
Open  = df[['Open']].iloc[0:35]
Market_Cap = df['Market Cap'].iloc[0:35]

In [ ]:
#Train model 
model.fit(Open, Market_Cap)

In [ ]:
#The model score almost 1 so that indicate the model is near to the truth
model.score(Open, Market_Cap)

In [ ]:
#Add a new column which is filled the missing data from model fit
open = df[['Open']]
Market_Cap_Predict = model.predict(open)
df['Market Cap Predict'] = Market_Cap_Predict
df.tail()